In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:



! pip install -U sentence-transformers -q

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

import re
import torch
import torch.nn.functional as F
from sentence_transformers import models as sentence_models
from transformers import BertTokenizer

## Models

In [ ]:
paraphrase = SentenceTransformer('paraphrase-distilroberta-base-v1')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model = SentenceTransformer('all-MiniLm-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
multi_lang_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/movies_dataset_saved.csv')

In [ ]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_ar,search,search_Mixt
0,1901,Kansas Saloon Smashers,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",يعمل النادل في صالون ، يقدم المشروبات للعملاء....,"Kansas Saloon Smashers, 1901, , American, : A ...","Kansas Saloon Smashers, 1901, , American, : A ..."
1,1901,Love by the Light of the Moon,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",القمر ، مطلي بوجه مبتسم معلق فوق حديقة في اللي...,"Love by the Light of the Moon, 1901, , America...","Love by the Light of the Moon, 1901, , America..."
2,1901,The Martyred Presidents,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",يتكون الفيلم ، الذي يزيد طوله عن دقيقة واحدة ،...,"The Martyred Presidents, 1901, , American, : T...","The Martyred Presidents, 1901, , American, : T..."
3,1901,"Terrible Teddy, the Grizzly King",American,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,تدوم 61 ثانية فقط وتتألف من طلقتين ، يتم وضع ا...,"Terrible Teddy, the Grizzly King, 1901, , Amer...","Terrible Teddy, the Grizzly King, 1901, , Amer..."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,وأقدم التكيف المعروف مع القصص الخيالية الكلاسي...,"Jack and the Beanstalk, 1902, , American, Geor...","Jack and the Beanstalk, 1902, , American, Geor..."


## Movies plots

### English plots

In [ ]:
movie_plots = {}
for idx, row in df.iterrows():
    plot_id = str(idx + 1)  # IDs start from 1
    plot_text =  row['Plot']
    movie_plots[plot_id] = plot_text
    if(idx == 100):
      break

In [ ]:
for idx, (plot_id, plot_text) in enumerate(movie_plots.items()):
    print(f"Movie ID: {plot_id}, Plot: {plot_text}")
    if idx == 21:
        break

Movie ID: 1, Plot: A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]
Movie ID: 2, Plot: The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.
Movie ID: 3, Plot: The 

### Arabic plots

In [ ]:
arabic_movie_plots = {}
for idx, row in df.iterrows():
    plot_id = str(idx + 1)  # IDs start from 1
    plot_text =  row['Plot_ar']
    arabic_movie_plots[plot_id] = plot_text
    if(idx == 100):
      break

In [ ]:
for idx, (plot_id, plot_text) in enumerate(arabic_movie_plots.items()):
    print(f"Movie ID: {plot_id}, Plot: {plot_text}")
    if idx == 21:
        break

Movie ID: 1, Plot: يعمل النادل في صالون ، يقدم المشروبات للعملاء.بعد أن يملأ دلو الرجل الأيرلندي النمطي مع البيرة ، انفجر كاري نيشن وأتباعها في الداخل.يعتنقون الرجل الأيرلندي ، وسحبوا قبعته على عينيه ثم يلقون البيرة على رأسه.ثم تبدأ المجموعة في تحطيم الشريط ، وتحطيم التركيبات ، والمرايا ، وكسر السجل النقدي.ثم يرش النادل مياه سيلتزر في وجه الأمة قبل أن تظهر مجموعة من رجال الشرطة ويأمر الجميع بالمغادرة. [1]
Movie ID: 2, Plot: القمر ، مطلي بوجه مبتسم معلق فوق حديقة في الليل.زوجان شابان يمشيان عبر سياج يتعلمان على حديدي وينظران.يبتسم القمر.يعتنقون ، وابتسامة القمر أكبر.ثم يجلسون على مقعد بواسطة شجرة.تم حظر وجهة نظر القمر ، مما تسبب له في عبوس.في المشهد الأخير ، كان الرجل مشجعًا للمرأة بقبعته لأن القمر قد غادر السماء وتجول فوق كتفها لرؤية كل شيء أفضل.
Movie ID: 3, Plot: يتكون الفيلم ، الذي يزيد طوله عن دقيقة واحدة ، من طلقتين.في البداية ، تجلس فتاة في قاعدة مذبح أو قبر ، مخبأ وجهها عن الكاميرا.في وسط المذبح ، تعرض بوابة المشاهدة صورًا لثلاثة رؤساء أمريكيين - أبراهام لينكولن ، وجيمس أ. غارفي

# Mixed plots

In [ ]:
mixed_movie_plots = {}
for idx, row in df.iterrows():
    plot_id = str(idx + 1)  # IDs start from 1
    plot_text =  row['search_Mixt']
    mixed_movie_plots[plot_id] = plot_text
    if(idx == 100):
      break


In [ ]:
data_plots = [plot for plot in movie_plots.values()]

In [ ]:
arabic_data_plots = [plot for plot in arabic_movie_plots.values()]

In [ ]:
mixed_data_plots = [plot for plot in mixed_movie_plots.values()]

## Embeddings

### English embeddings

In [ ]:
embeddings = model.encode(data_plots)


### Arabic embeddings

In [ ]:
arabic_embeddings = model.encode(arabic_data_plots)


# Mixed Embeddings

In [ ]:
mixed_embeddings = model.encode(mixed_data_plots)

## Queries for testing

### English queries

In [ ]:
# First List: First Queries for Each Movie
first_queries = [
    "Saloon chaos erupts as Irish man's bucket is filled with beer.",
    "Romantic couple enchanted by the moon's smile.",
    "Portraits of assassinated presidents on display.",
    "Vice-president Roosevelt hunts in the woods.",
    "Jack trades his cow for magical beans.",
    "Alice follows the white rabbit down the rabbit-hole.",
    "Bandits stage a daring train robbery.",
    "Family's move to the suburbs turns chaotic.",
    "Bandits plan and execute a train robbery.",
    "Santa Claus prepares for his Christmas journey.",
    "Rarebit fiend hallucinates after eating Welsh rarebit.",
    "Thugs create a hold-up on a train in the Rockies.",
    "Irish villagers celebrate the defeat of Captain Clearfield.",
    "Daniel Boone's daughter is abducted by Indians.",
    "Mr. Brown's drunken misadventures at a baseball game.",
    "A black woman's comedic experience with laughing gas.",
    "Dollie is kidnapped by gypsies during an outing.",
    "Thug accosts a girl, leading to a rescue by a man.",
    "Young couple's elopement plans are thwarted by a thief.",
    "A white girl rejects an Indian brave's proposal."
]



In [ ]:
# Second List: Second Queries for Each Movie
second_queries = [
    "Carrie Nation's unexpected visit to the saloon leads to chaos.",
    "Park bench scene under the moon's watchful eye.",
    "Assassin kneels before Lady Justice.",
    "Press agent documents Roosevelt's hunting expedition.",
    "Jack climbs the beanstalk and encounters the giant.",
    "Alice attends the Mad Tea-Party.",
    "Posse pursues the fleeing bandits.",
    "Wife's violent outburst leads to her arrest.",
    "Police pursue the fleeing robbers.",
    "Santa delivers presents to children on Christmas Eve.",
    "The rarebit fiend's wild dreams come to life.",
    "Robbers systematically rob the train's passengers.",
    "Terence O'More saves the village from oppression.",
    "Boone embarks on a perilous rescue mission.",
    "Mr. Brown's intoxicated perception of the baseball game.",
    "The contagious laughter spreads to others in the neighborhood.",
    "Rescue party searches for Dollie in the mountains.",
    "Girl and rescuer are kidnapped by the thugs.",
    "Thief hides in the couple's trunk to steal their belongings.",
    "Indian brave comes to the girl's defense when she is abducted."
]

In [ ]:
# Third List: Third Queries for Each Movie
third_queries = [
    "Police intervention ends the saloon altercation.",
    "The moon's frown disappoints the couple.",
    "The film's two-shot composition.",
    "Roosevelt's triumphant display of his trophy.",
    "The fairy reveals Jack's royal destiny.",
    "Alice's encounter with the Queen's procession.",
    "Final shootout and recovery of stolen loot.",
    "The quiet suburban life turns tumultuous.",
    "Final capture of the bedraggled robbers.",
    "Children awaken to find their Christmas presents.",
    "The fiend awakens from his surreal dream.",
    "Final confrontation and capture of the robbers.",
    "O'More's heroism leads to a joyful celebration.",
    "Boone's epic battle against the Indian chief.",
    "Mr. Brown's wife confronts the friend responsible for his intoxication.",
    "Police officers join in the hilarity caused by laughing gas.",
    "Dollie's dramatic rescue from the gypsies.",
    "Final confrontation and rescue from the thugs.",
    "The couple's elopement turns into a calamitous adventure.",
    "Girl is reunited safely with her parents after a perilous ordeal."
]

### Arabic queries

In [ ]:
# First List: First Queries for Each Movie
arabic_first_queries = [
    "يحدث فوضى في الصالون عندما يمتلئ دلو الرجل الأيرلندي بالبيرة.",
    "زوجان رومانسيان يسحرهما ابتسامة القمر.",
    "صور الرؤساء المغتالين معروضة.",
    "نائب الرئيس روزفلت يصطاد في الغابة.",
    "يقايض جاك بقرته بالفاصوليا السحرية.",
    "أليس تتبع الأرنب الأبيض إلى داخل الجحر.",
    "قطاع الطرق ينفذون عملية سطو جريئة على القطار.",
    "انتقال العائلة إلى الضواحي يتحول إلى فوضى.",
    "قطاع الطرق يخططون وينفذون عملية سطو على القطار.",
    "سانتا كلوز يستعد لرحلته في عيد الميلاد.",
    "مدمن الويلزية يهلوس بعد تناول الويلزية النادرة.",
    "الأشقياء ينفذون عملية سطو على قطار في جبال الروكي.",
    "القرويون الأيرلنديون يحتفلون بهزيمة الكابتن كليرفيلد",
    "ابنة دانيال بون تُختطف من قبل الهنود.",
    "مغامرات السيد براون السكرانة في مباراة بيسبول.",
    "تجربة امرأة سوداء الكوميدية مع غاز الضحك.",
    "يتم اختطاف دوللي من قبل الغجر أثناء نزهة.",
    "مجرم يعترض فتاة، مما يؤدي إلى إنقاذها بواسطة رجل.",
    "خطط هروب الزوجين الشابين تُحبط بواسطة لص.",
    "فتاة بيضاء ترفض عرض زواج من شجاع هندي."
]

# Second List: Second Queries for Each Movie
arabic_second_queries = [
    "تؤدي زيارة كاري نيشن  غير المتوقعة إلى الصالون إلى الفوضى",
    "مشهد مقعد الحديقة تحت عين القمر الساهرة",
    "القاتل يركع أمام  العدالة.",
    "وكيل الصحافة يوثق رحلة صيد روزفلت.",
    "جاك يتسلق شجرة الفاصولياء ويواجه العملاق.",
    "أليس تحضر حفل الشاي المجنون.",
    "مجموعة من الرجال يطاردون قطاع الطرق الهاربين.",
    "انفجار الزوجة العنيف يؤدي إلى اعتقالها.",
    "الشرطة تطارد اللصوص الفارين.",
    "سانتا يوزع الهدايا على الأطفال في ليلة عيد الميلاد.",
    "أحلام مدمن الويلزية البرية تتحقق.",
    "اللصوص يسرقون ركاب القطار بشكل منهجي.",
    "تيرينس أو'مور ينقذ القرية من القمع.",
    "بون ينطلق في مهمة إنقاذ محفوفة بالمخاطر.",
    "إدراك السيد براون للمباراة وهو في حالة سكر.",
    "الضحك المعدي ينتشر للآخرين في الحي.",
    "فريق الإنقاذ يبحث عن دولي في الجبال.",
    "يتم اختطاف الفتاة والمنقذ من قبل المجرمين.",
    "يختبئ اللص في حقيبة الزوجين ليسرق ممتلكاتهما.",
    "الشجاع الهندي يأتي للدفاع عن الفتاة عندما تُختطف."
]

# Third List: Third Queries for Each Movie
arabic_third_queries = [
    "تدخل الشرطة لانهاء الشجار في الصالون.",
    "تجهم القمر يخيب آمال الزوجين.",
    "تكوين المشهد ذو اللقطتين في الفيلم.",
    "عرض روزفلت المنتصر لجوائزه.",
    "الجنية تكشف عن مصير جاك الملكي.",
    "لقاء أليس مع موكب الملكة.",
    "المواجهة النهائية واستعادة المسروقات.",
    "الحياة الهادئة في الضواحي تتحول إلى مضطربة.",
    "القبض النهائي على اللصوص المتهالكين.",
    "الأطفال يستيقظون ليجدوا هدايا عيد الميلاد الخاصة بهم.",
    "المدمن يستيقظ من حلمه السريالي.",
    "المواجهة النهائية والقبض على اللصوص.",
    "بطولة أو'مور تؤدي إلى احتفال سعيد.",
    "معركة بون الملحمية ضد زعيم الهنود.",
    "زوجة السيد براون تواجه الصديق المسؤول عن سكره.",
    "الشرطة تشارك في الهستيريا التي سببها غاز الضحك.",
    "الإنقاذ الدرامي لدولي من الغجر.",
    "المواجهة النهائية والإنقاذ من المجرمين.",
    "يتحول هروب الزوجين إلى مغامرة كارثية.",
    "تعود الفتاة بأمان إلى والديها بعد محنة خطيرة."
]


In [ ]:
mixed_first_queries = [
    "Saloon chaos erupts when يمتلئ دلو الرجل الأيرلندي بالبيرة.",
    "Romantic couple enchanted by ابتسامة القمر.",
    "Portraits of الرؤساء المغتالين on display.",
    "Vice-president Roosevelt hunts in الغابة.",
    "Jack trades his cow for الفاصوليا السحرية.",
    "Alice follows the white rabbit إلى داخل الجحر.",
    "Bandits stage a daring عملية سطو جريئة على القطار.",
    "Family's move to the suburbs يتحول إلى فوضى.",
    "Bandits plan and ينفذون عملية سطو على القطار.",
    "Santa Claus prepares for رحلته في عيد الميلاد.",
    "Rarebit fiend hallucinates after تناول الويلزية النادرة.",
    "Thugs create a hold-up on a train في جبال الروكي.",
    "Irish villagers celebrate هزيمة الكابتن كليرفيلد.",
    "Daniel Boone's daughter is تُختطف من قبل الهنود.",
    "Mr. Brown's drunken misadventures at مباراة بيسبول.",
    "A black woman's comedic experience with غاز الضحك.",
    "Dollie is kidnapped by الغجر أثناء نزهة.",
    "Thug accosts a girl, leading to إنقاذها بواسطة رجل.",
    "Young couple's elopement plans تُحبط بواسطة لص.",
    "A white girl rejects عرض زواج من شجاع هندي.",
]


mixed_second_queries = [

    "Carrie Nation's unexpected visit to the saloon تؤدي إلى الفوضى.",
    "مشهد مقعد الحديقة under the moon's watchful eye.",
    "The assassin kneels before العدالة.",
    "Press agent documents رحلة صيد روزفلت.",
    "Jack climbs the beanstalk and يواجه العملاق.",
    "Alice attends the Mad حفل الشاي المجنون.",
    "Posse pursues قطاع الطرق الهاربين.",
    "Wife's violent outburst يؤدي إلى اعتقالها.",
    "Police pursue the اللصوص الفارين.",
    "Santa delivers presents to الأطفال في ليلة عيد الميلاد.",
    "The rarebit fiend's wild dreams تتحقق.",
    "Robbers systematically يسرقون ركاب القطار.",
    "Terence O'More saves القرية من القمع.",
    "Boone embarks on مهمة إنقاذ محفوفة بالمخاطر.",
    "Mr. Brown's intoxicated perception of المباراة.",
    "The contagious laughter spreads to الآخرين في الحي.",
    "Rescue party searches for دوللي في الجبال.",
    "Girl and rescuer are يتم اختطافهما من قبل المجرمين.",
    "Thief hides in the حقيبة الزوجين.",
    "Indian brave comes to الدفاع عن الفتاة عندما تُختطف.",
]

mixed_third_queries = [
    # Movie 1: Kansas Saloon Smashers
    "Police intervention ends الشجار في الصالون.",

    # Movie 2: Love by the Light of the Moon
    "The moon's frown يخيب آمال الزوجين.",

    # Movie 3: The Martyred Presidents
    "تكوين المشهد ذو اللقطتين showcases the film's composition.",

    # Movie 4: Terrible Teddy, the Grizzly King
    "Roosevelt's triumphant display of جوائزه.",

    # Movie 5: Jack and the Beanstalk
    "The fairy reveals مصير جاك الملكي.",

    # Movie 6: Alice in Wonderland
    "Alice's encounter with موكب الملكة.",

    # Movie 7: The Great Train Robbery
    "Final shootout and استعادة المسروقات.",

    # Movie 8: The Suburbanite
    "The quiet suburban life تتحول إلى مضطربة.",

    # Movie 9: The Little Train Robbery
    "Final capture of the اللصوص المتهالكين.",

    # Movie 10: The Night Before Christmas
    "Children awaken to find هدايا عيد الميلاد الخاصة بهم.",

    # Movie 11: Dream of a Rarebit Fiend
    "The fiend awakens from حلمه السريالي.",

    # Movie 12: From Leadville to Aspen: A Hold-Up in the Rockies
    "Final confrontation and القبض على اللصوص.",

    # Movie 13: Kathleen Mavourneen
    "O'More's heroism leads to احتفال سعيد.",

    # Movie 14: Daniel Boone
    "Boone's epic battle against زعيم الهنود.",

    # Movie 15: How Brown Saw the Baseball Game
    "Mr. Brown's wife confronts الصديق المسؤول عن سكره.",

    # Movie 16: Laughing Gas
    "Police officers join in the الهستيريا التي سببها غاز الضحك.",

    # Movie 17: The Adventures of Dollie
    "Dollie's dramatic rescue from الغجر.",

    # Movie 18: The Black Viper
    "Final confrontation and الإنقاذ من المجرمين.",

    # Movie 19: A Calamitous Elopement
    "The couple's elopement turns into مغامرة كارثية.",

    # Movie 20: The Call of the Wild
    "Girl is reunited safely with والديها بعد محنة خطيرة."
]

## Expected Results

### English Expected Results

In [ ]:
expected_results_per_query = {
    # Movie 1
    "Saloon chaos erupts as Irish man's bucket is filled with beer.": {"1", "3", "93", "7", "82", "40"},

    # Movie 2
    "Romantic couple enchanted by the moon's smile.": {"1", "22", "8", "45", "66", "71"},

    # Movie 3
    "Portraits of assassinated presidents on display.": {"3", "75", "4", "22", "72", "34"},

    # Movie 4
    "Vice-president Roosevelt hunts in the woods.": {"4", "24", "74", "5", "16", "13"},

    # Movie 5
    "Jack trades his cow for magical beans.": {"5", "6", "88", "24", "54", "4"},

    # Movie 6
    "Alice follows the white rabbit down the rabbit-hole.": {"9", "46", "97", "93", "6", "34"},

    # Movie 7
    "Bandits stage a daring train robbery.": {"7", "9", "13", "56", "21", "30"},

    # Movie 8
    "Family's move to the suburbs turns chaotic.": {"81", "34", "97", "22", "85", "88"},

    # Movie 9
    "Bandits plan and execute a train robbery.": {"7", "5", "13", "56", "30", "21"},

    # Movie 10
    "Santa Claus prepares for his Christmas journey.": {"10", "6", "27", "29", "28", "11"},

    # Movie 11
    "Rarebit fiend hallucinates after eating Welsh rarebit.": {"12", "37", "61", "75", "38", "82"},

    # Movie 12
    "Thugs create a hold-up on a train in the Rockies.": {"13", "21", "7", "9", "30", "64"},

    # Movie 13
    "Irish villagers celebrate the defeat of Captain Clearfield.": {"14", "40", "1", "76", "67", "15"},

    # Movie 14
    "Daniel Boone's daughter is abducted by Indians.": {"16", "42", "41", "23", "56", "63"},

    # Movie 15
    "Mr. Brown's drunken misadventures at a baseball game.": {"17", "93", "47", "50", "1", "49"},

    # Movie 16
    "A black woman's comedic experience with laughing gas.": {"18", "19", "66", "8", "93", "33"},

    # Movie 17
    "Dollie is kidnapped by gypsies during an outing.": {"20", "45", "100", "41", "9", "78"},

    # Movie 18
    "Thug accosts a girl, leading to a rescue by a man.": {"21", "51", "78", "9", "79", "64"},

    # Movie 19
    "Young couple's elopement plans are thwarted by a thief.": {"22", "78", "64", "31", "8", "97"},

    # Movie 20
    "A white girl rejects an Indian brave's proposal.": {"23", "31", "16", "76", "44", "45"},
}


### Arabic Expected Results

In [ ]:
arabic_expected_results_per_query = {
    # Movie 1
    "يحدث فوضى في الصالون عندما يمتلئ دلو الرجل الأيرلندي بالبيرة.": {"1", "13", "93", "7", "82", "40"},

    # Movie 2
    "زوجان رومانسيان يسحرهما ابتسامة القمر.": {"2", "22", "89", "45", "66", "71"},

    # Movie 3
    "صور الرؤساء المغتالين معروضة.": {"3", "75", "4", "2", "72", "34"},

    # Movie 4
    "نائب الرئيس روزفلت يصطاد في الغابة.": {"4", "24", "74", "25", "16", "13"},

    # Movie 5
    "يقايض جاك بقرته بالفاصوليا السحرية.": {"5", "6", "88", "24", "54", "46"},

    # Movie 6
    "أليس تتبع الأرنب الأبيض إلى داخل الجحر.": {"94", "46", "97", "93", "6", "34"},

    # Movie 7
    "قطاع الطرق ينفذون عملية سطو جريئة على القطار.": {"7", "9", "13", "56", "21", "30"},

    # Movie 8
    "انتقال العائلة إلى الضواحي يتحول إلى فوضى.": {"8", "34", "97", "22", "85", "88"},

    # Movie 9
    "قطاع الطرق يخططون وينفذون عملية سطو على القطار.": {"7", "9", "13", "56", "30", "21"},

    # Movie 10
    "سانتا كلوز يستعد لرحلته في عيد الميلاد.": {"10", "26", "27", "29", "28", "11"},

    # Movie 11
    "مدمن الويلزية يهلوس بعد تناول الويلزية النادرة.": {"12", "37", "61", "75", "38", "82"},

    # Movie 12
    "الأشقياء ينفذون عملية سطو على قطار في جبال الروكي.": {"13", "21", "7", "9", "30", "64"},

    # Movie 13
    "القرويون الأيرلنديون يحتفلون بهزيمة الكابتن كليرفيلد": {"14", "40", "1", "76", "67", "15"},

    # Movie 14
    "ابنة دانيال بون تُختطف من قبل الهنود.": {"16", "42", "41", "23", "56", "63"},

    # Movie 15
    "مغامرات السيد براون السكرانة في مباراة بيسبول.": {"17", "93", "47", "50", "1", "49"},

    # Movie 16
    "تجربة امرأة سوداء الكوميدية مع غاز الضحك.": {"18", "19", "66", "8", "93", "33"},

    # Movie 17
    "يتم اختطاف دوللي من قبل الغجر أثناء نزهة.": {"20", "45", "100", "41", "9", "78"},

    # Movie 18
    "مجرم يعترض فتاة، مما يؤدي إلى إنقاذها بواسطة رجل.": {"21", "51", "78", "9", "79", "64"},

    # Movie 19
    "خطط هروب الزوجين الشابين تُحبط بواسطة لص.": {"22", "78", "64", "31", "8", "97"},

    # Movie 20
    "فتاة بيضاء ترفض عرض زواج من شجاع هندي.": {"23", "31", "16", "76", "44", "45"},
}


In [ ]:
mixed_expected_results_per_query = {
    # Movie 1
    "Saloon chaos erupts when يمتلئ دلو الرجل الأيرلندي بالبيرة.": {"1", "13", "93", "7", "82", "40"},

    # Movie 2
    "Romantic couple enchanted by ابتسامة القمر.": {"2", "22", "89", "45", "66", "71"},

    # Movie 3
    "Portraits of الرؤساء المغتالين on display.": {"3", "75", "4", "2", "72", "34"},

    # Movie 4
    "Vice-president Roosevelt hunts in الغابة.": {"4", "24", "74", "25", "16", "13"},

    # Movie 5
    "Jack trades his cow for الفاصوليا السحرية.": {"5", "6", "88", "24", "54", "46"},

    # Movie 6
    "Alice follows the white rabbit إلى داخل الجحر.": {"94", "46", "97", "93", "6", "34"},

    # Movie 7
    "Bandits stage a daring عملية سطو جريئة على القطار.": {"7", "9", "13", "56", "21", "30"},

    # Movie 8
    "Family's move to the suburbs يتحول إلى فوضى.": {"8", "34", "97", "22", "85", "88"},

    # Movie 9
    "Bandits plan and ينفذون عملية سطو على القطار.": {"7", "9", "13", "56", "30", "21"},

    # Movie 10
    "Santa Claus prepares for رحلته في عيد الميلاد.": {"10", "26", "27", "29", "28", "11"},

    # Movie 11
    "Rarebit fiend hallucinates after تناول الويلزية النادرة.": {"12", "37", "61", "75", "38", "82"},

    # Movie 12
    "Thugs create a hold-up on a train في جبال الروكي.": {"13", "21", "7", "9", "30", "64"},

    # Movie 13
    "Irish villagers celebrate هزيمة الكابتن كليرفيلد.": {"14", "40", "1", "76", "67", "15"},

    # Movie 14
    "Daniel Boone's daughter is تُختطف من قبل الهنود.": {"14", "42", "41", "23", "56", "63"},

    # Movie 15
    "Mr. Brown's drunken misadventures at مباراة بيسبول.": {"17", "93", "47", "50", "1", "49"},

    # Movie 16
    "A black woman's comedic experience with غاز الضحك.": {"18", "19", "66", "8", "93", "33"},

    # Movie 17
    "Dollie is kidnapped by الغجر أثناء نزهة.": {"20", "45", "100", "41", "9", "78"},

    # Movie 18
    "Thug accosts a girl, leading to إنقاذها بواسطة رجل." : {"21", "51", "78", "9", "79", "64"},

    # Movie 19
    "Young couple's elopement plans تُحبط بواسطة لص.": {"22", "78", "64", "31", "8", "97"},

    # Movie 20
    "A white girl rejects عرض زواج من شجاع هندي." : {"23", "31", "16", "76", "44", "45"},
}

## Semantic Search

In [ ]:
def semantic_search(query, movie_plots, embeddings, model):
    """
    Perform a semantic search to find the most similar movie plots to the given query.

    Parameters:
    - query (str): The search query.
    - movie_plots (dict): A dictionary where keys are movie IDs and values are movie plot descriptions.
    - embeddings (np.ndarray): A 2D array where each row is the embedding of a movie plot.
    - model: A pre-trained model used to encode the query into an embedding.

    Returns:
    - search_results (list): A list of tuples containing movie ID, plot description, and similarity score.
    """

    # Encode the query into an embedding
    query_embedding = model.encode([query])[0]

    # Calculate cosine similarities between the query embedding and all movie embeddings
    similarities = cosine_similarity([query_embedding], embeddings)[0]

    # Sort the results by similarity in descending order
    sorted_results = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)[:7]

    # Format the search results
    search_results = [(str(idx+1), movie_plots[str(idx+1)], similarity) for idx, similarity in sorted_results]

    return search_results


## Evaluation

In [ ]:
def evaluate_search_engine(queries, ground_truth, movie_plots, embeddings, model):
    precision_list = []
    recall_list = []
    f1_score_list = []

    for query in queries:
        relevant_movies = ground_truth[query]

        search_results = semantic_search(query, movie_plots, embeddings, model)

        retrieved_movies = set([movie_id for movie_id, _, _ in search_results])

        true_positives = len(relevant_movies.intersection(retrieved_movies))
        false_positives = len(retrieved_movies - relevant_movies)
        false_negatives = len(relevant_movies - retrieved_movies)

        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1_score)

    average_precision = sum(precision_list) / len(precision_list)
    average_recall = sum(recall_list) / len(recall_list)
    average_f1_score = sum(f1_score_list) / len(f1_score_list)

    return average_precision, average_recall, average_f1_score


## English queries evaluation

In [ ]:
average_precision, average_recall, average_f1_score = evaluate_search_engine(first_queries, expected_results_per_query, movie_plots, embeddings, model)

print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1-Score: {average_f1_score:.2f}")

Average Precision: 0.79
Average Recall: 0.93
Average F1-Score: 0.85


## Arabic queries evaluation

In [ ]:
average_precision, average_recall, average_f1_score = evaluate_search_engine(arabic_first_queries, arabic_expected_results_per_query, arabic_movie_plots, arabic_embeddings, multi_lang_model)

print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1-Score: {average_f1_score:.2f}")

Average Precision: 0.41
Average Recall: 0.48
Average F1-Score: 0.45


Mixed querioes evaluation

In [ ]:
average_precision, average_recall, average_f1_score = evaluate_search_engine(mixed_first_queries, mixed_expected_results_per_query, mixed_movie_plots, mixed_embeddings, multi_lang_model)
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1-Score: {average_f1_score:.2f}")

Average Precision: 0.42
Average Recall: 0.49
Average F1-Score: 0.45


# Paraphrase model Evaluation

In [ ]:
paraphrase_embeddings = paraphrase.encode(data_plots)

In [ ]:
average_precision, average_recall, average_f1_score = evaluate_search_engine(first_queries, expected_results_per_query, movie_plots, paraphrase_embeddings, paraphrase)
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1-Score: {average_f1_score:.2f}")


Average Precision: 0.37
Average Recall: 0.43
Average F1-Score: 0.40
